In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import sparse 
from sklearn.metrics.pairwise import cosine_similarity
amazon = pd.read_csv('Amazon.csv')
amazon.info()
amazon.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4848 entries, 0 to 4847
Columns: 207 entries, user_id to Movie206
dtypes: float64(206), object(1)
memory usage: 7.7+ MB


,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
amazon = amazon.melt(id_vars=['user_id'], var_name='name', value_name='rating')
amazon_pivot = amazon.pivot_table(index=['user_id'], columns=['name'], values='rating')
amazon_pivot.head()

name,Movie1,Movie10,Movie100,Movie101,Movie102,Movie103,Movie104,Movie105,Movie106,Movie107,...,Movie90,Movie91,Movie92,Movie93,Movie94,Movie95,Movie96,Movie97,Movie98,Movie99
user_id,,,,,,,,,,,,,,,,,,,,,
A0047322388NOTO4N8SKD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A00473363TJ8YSZ3YAGG9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1004AX2J2HXGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100CQXJ6D44T9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100Z2S0880G9A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
amazon_normalized = amazon_pivot.apply(lambda x: (x-np.min(x))/(np.max(x)-np.min(x)), axis=1)
amazon_normalized.fillna(0, inplace=True)
amazon_normalized = amazon_normalized.T
amazon_normalized = amazon_normalized.loc[:, (amazon_normalized !=0).any(axis=0)]
amazon_normalized.head()

user_id,A140XH16IKR4B0,A17LS4IPCWWZU4,A17UAYGNHJ5QJL,A1GNZ19FWDR0QU,A1IG5SXEQ5OJ52,A1L9O8FL2REQTE,A1RG3EPDJD26UU,A1ZAN7KBME97B3,A25KKXUQQ0OLWB,A27TQA0XCZUXZY,...,A3EVDRB2NK2UHS,A50JHJ9MLSZAN,A6EKCSZ2XWDGV,A6GMEO3VRY51S,A9Q28YTLYREO7,ACIBQ6BQ6AWEV,ADO97N7YIO14S,AEJAFBCTVLV9Q,AP3B615GM191G,AZ0MUPV0DU1R9
name,,,,,,,,,,,,,,,,,,,,,
Movie1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Movie10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Movie100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Movie101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Movie102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
amazon_sparse = sp.sparse.csr_matrix(amazon_normalized.values)
amazon_sparse

<206x27 sparse matrix of type '<class 'numpy.float64'>'
	with 35 stored elements in Compressed Sparse Row format>

In [8]:
item_similarity = cosine_similarity(amazon_sparse)
item_sim_df = pd.DataFrame(item_similarity, index=amazon_normalized.index, columns=amazon_normalized.index)
item_sim_df.head()

name,Movie1,Movie10,Movie100,Movie101,Movie102,Movie103,Movie104,Movie105,Movie106,Movie107,...,Movie90,Movie91,Movie92,Movie93,Movie94,Movie95,Movie96,Movie97,Movie98,Movie99
name,,,,,,,,,,,,,,,,,,,,,
Movie1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Movie10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Movie100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Movie101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Movie102,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.5,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [9]:
def top_movie(movie_name):
    for item in item_sim_df.sort_values(by = movie_name, ascending = False).index[1:11]:
        print('Similar movie:{}'.format(item))
top_movie("Movie102")

Similar movie:Movie95
Similar movie:Movie86
Similar movie:Movie92
Similar movie:Movie1
Similar movie:Movie31
Similar movie:Movie32
Similar movie:Movie33
Similar movie:Movie34
Similar movie:Movie35
Similar movie:Movie36


In [10]:
user_similarity = cosine_similarity(amazon_sparse.T)
user_sim_df = pd.DataFrame(user_similarity, index = amazon_normalized.columns, columns = amazon_normalized.columns)
user_sim_df.head()

user_id,A140XH16IKR4B0,A17LS4IPCWWZU4,A17UAYGNHJ5QJL,A1GNZ19FWDR0QU,A1IG5SXEQ5OJ52,A1L9O8FL2REQTE,A1RG3EPDJD26UU,A1ZAN7KBME97B3,A25KKXUQQ0OLWB,A27TQA0XCZUXZY,...,A3EVDRB2NK2UHS,A50JHJ9MLSZAN,A6EKCSZ2XWDGV,A6GMEO3VRY51S,A9Q28YTLYREO7,ACIBQ6BQ6AWEV,ADO97N7YIO14S,AEJAFBCTVLV9Q,AP3B615GM191G,AZ0MUPV0DU1R9
user_id,,,,,,,,,,,,,,,,,,,,,
A140XH16IKR4B0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.894427,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.707107,0.0
A17LS4IPCWWZU4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
A17UAYGNHJ5QJL,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
A1GNZ19FWDR0QU,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.894427,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.707107,0.0
A1IG5SXEQ5OJ52,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [11]:
def top_users(user):  
    sim_values = user_sim_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:11]
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:11]
    zipped = zip(sim_users, sim_values)
    for user, sim in zipped:
        print('User #{0}, Similarity value: {1:.2f}'.format(user, sim))
top_users('A140XH16IKR4B0')

User #A1GNZ19FWDR0QU, Similarity value: 1.00
User #AEJAFBCTVLV9Q, Similarity value: 1.00
User #ACIBQ6BQ6AWEV, Similarity value: 1.00
User #A1ZAN7KBME97B3, Similarity value: 1.00
User #A27TQA0XCZUXZY, Similarity value: 1.00
User #A9Q28YTLYREO7, Similarity value: 1.00
User #A25KKXUQQ0OLWB, Similarity value: 0.89
User #AP3B615GM191G, Similarity value: 0.71
User #A3CWH6VKCTJAD, Similarity value: 0.00
User #ADO97N7YIO14S, Similarity value: 0.00
